# Day 22 - Modular arithmetic and linear congruential generator

The card shuffling has hints of [modular arithmetic](https://en.wikipedia.org/wiki/Modular_arithmetic). We have reversing, rotation and dealing out cards in a new order that puts cards in locations ruled by the modulus of the length of the deck.


In [1]:
from __future__ import annotations

import re
from typing import Any, Callable, Dict, List, Sequence

Deck = List[int]
Shuffle = Callable[[Deck, int], Deck]


def _deal_with_increment(deck: Sequence[int], num: int, *args: Any) -> Sequence[int]:
    length = len(deck)
    result = [0] * len(deck)
    for i, v in enumerate(deck):
        result[i * num % length] = v
    return result


OPS: Dict[str, Shuffle] = {
    "deal into new stack": lambda deck, num: deck[::-1],
    "cut": lambda deck, num: deck[num:] + deck[:num],
    "deal with increment": _deal_with_increment,
}


def reshuffle(
    instructions: Sequence[str], deck_size: int = 10007, ops: Dict[str, Shuffle] = None
) -> Sequence[int]:
    if ops is None:
        ops = OPS
    deck = list(range(deck_size))
    for instruction in instructions:
        try:
            instr, num, *_ = re.split(r"\s*(-?\d+)", instruction)
            num = int(num)
        except ValueError:
            instr, num = instruction, 0
        deck = ops[instr](deck, num)
    return deck


assert OPS["deal into new stack"](list(range(10)), 0) == list(range(9, -1, -1))
assert OPS["cut"](list(range(10)), 3) == [3, 4, 5, 6, 7, 8, 9, 0, 1, 2]
assert OPS["cut"](list(range(10)), -4) == [6, 7, 8, 9, 0, 1, 2, 3, 4, 5]
assert OPS["deal with increment"](list(range(10)), 3) == [0, 7, 4, 1, 8, 5, 2, 9, 6, 3]

part1_tests = {
    "deal with increment 7\ndeal into new stack\ndeal into new stack": [
        0,
        3,
        6,
        9,
        2,
        5,
        8,
        1,
        4,
        7,
    ],
    "cut 6\ndeal with increment 7\ndeal into new stack": [3, 0, 7, 4, 1, 8, 5, 2, 9, 6],
    "deal with increment 7\ndeal with increment 9\ncut -2": [
        6,
        3,
        0,
        7,
        4,
        1,
        8,
        5,
        2,
        9,
    ],
    (
        "deal into new stack\ncut -2\ndeal with increment 7\ncut 8\ncut -4\ndeal with increment 7\n"
        "cut 3\ndeal with increment 9\ndeal with increment 3\ncut -1"
    ): [9, 2, 5, 8, 1, 4, 7, 0, 3, 6],
}
for instr, expected in part1_tests.items():
    assert reshuffle(instr.splitlines(), 10) == expected

In [2]:
import aocd

data = aocd.get_data(day=22, year=2019)

In [3]:
print("Part 1:", reshuffle(data.splitlines()).index(2019))

Part 1: 7171


## Part 2

Of course things had to be scaled up, _and_ we now have to find the value at a position, not the position of a value, so we are asked to _reverse the problem_. This is a maths problem, not a list reshuffling problem, now!

We can calculate value $v$ for a given index $i$ by figuring out what each operation does to the positions of cards, and reversing that operation. If the deck size is $N$, and the numeric value of a rule is $k$, then we can use the following functions; these calculate the index the value started at before the operation:

| Operation               | Function &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;                                            |
| :---------------------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| deal into new stack     | $v(i, k, N) = (N - 1 - i)\bmod{N}$ <sup><a href="#notes1" title="the modulus is redundant but serves to illustrate all formulas here are modulo N">(1)</a></sup> |
| cut $k$                 | $v(i, k, N) = (i + k + N)\bmod{N}$ <sup><a href="#notes2" title="+ N is used to avoiding having to handle the modulus of negative numbers">(2)</a></sup>         |
| deal with increment $k$ | $v(i, k, N) = modinv(k, N) \cdot i \bmod N$                                                                                                                      |

_Notes_

1. <a name="notes1"></a> the modulus is redundant but serves to illustrate all formulas here are modulo N
2. <a name="notes2"></a> $+ N$ is used to avoiding having to handle the modulus of negative numbers

The last function refers to the [_modular multiplicative inverse_](https://en.wikipedia.org/wiki/Modular_multiplicative_inverse) function, which works for values that are coprime with $N$. Given that [119315717514047 is a prime number](https://www.wolframalpha.com/input/?i=119315717514047), all indices in the deck are coprime except 0, and index 0 always stays at 0.

$modinv(a, m)$ gives you the number $x$ for which $ax\bmod m$ is equal to 1, but because in our case $m$ is prime and so there are no common divisors between $a$ and $m$, $x$ is equal to $a^{-1} \bmod m$ (because $a^{-1}$ is the same as $\frac{1}{a}$, and $a \cdot a^{-1} \equiv 1 \pmod m$ is $\frac{a}{a} \equiv 1 \pmod m$ as the only solution when $m$ is a prime number). Python has a really helpful built-in function for this, [`pow()`](https://docs.python.org/3/library/functions.html#pow), which gives you the outcome of $a^b \bmod m$, so as long as $m$ is a prime number, we can define $modinv(a, m)$ as `pow(a, -1, m)`.

We still have to apply these operations a _very large number of times_, so we should try to find a way to combine and simplify these functions. The first thing I note is that all we are dealing with (ignoring the modulus operations for brief moment) are simple multiplications and additions, a full run of the input rules can be expressed as a single function $f_{steps}(i) = ai + b \bmod N$, with $a$ and $b$ integers.

That, right there, is the [definition of a linear congruential generator](https://en.wikipedia.org/wiki/Linear_congruential_generator), basically a simple random number generator! And we can [_skip_ iterations of a LCG](https://www.nayuki.io/page/fast-skipping-in-a-linear-congruential-generator) by calculating the difference between two iterations of $f_{steps}$.

Applying $f_{steps}(i)$ to itself (as we are asked to do), produces a series like this (using $a^0$ instead of 1 and $a^1$ instead of $a$ to illustrate that we are dealing with increasing exponents):

$$
\begin{align*}
&a^{1}i + a^{0}b \\
&a^{2}i + a^{1}b + a^{0}b \\
&a^{3}i + a^{2}b + a^{1}b + a^{0}b \\
&a^{4}i + a^{3}b + a^{2}b + a^{1}b + a^{0}b \\
&\vdots \\
&a^{R}i + (a^{R-1} + a^{R-2} + \cdots + a^{0})b \\
\end{align*}
$$

where $R$ is the number of repetitions of our input rules.

There is a [geometric series](https://en.wikipedia.org/wiki/Geometric_series) in there for the exponents of $a$ when multiplying $b$, and so [can be expressed as](https://en.wikipedia.org/wiki/Geometric_series#Formula)

$$
a^{R}i + \frac{1 - a^R}{1 - a}b
$$

Those numbers are going to get big, _quickly_, which is why the puzzle description warns you that

> You'll need to be careful, though - one wrong move with this many cards and you might overflow your entire ship!

Luckily, we temporarily ignored the modulus, and Python's `pow()` lets us take the modulus along when applying the exponents. For that to work we need to be careful, because our formula involves division; adding $\bmod N$ back in gives us:

$$
\left((a^R \bmod N)i + \frac{1 - a^R}{1 - a}b\right) \bmod N
$$

and because `N` is a prime number, we can further simplify that to:

$$
\left((a^R \bmod N)i + (1 - a^R)modinv(1 - a, N)b\right) \bmod N
$$

We can calculate $a$ and $b$ by separating out the multiplier and the addition from our earlier table of $v(i, k, N)$ functions:

| Operation               | function &emsp;&emsp;&emsp;&emsp;&emsp; | a &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; | b &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; |
| :---------------------- | :-------------------------------------- | :------------------------------------------------------------- | :------------------------------------------------------------- |
| deal into new stack     | $N - 1 - i$                             | $-a$                                                           | $b - a$                                                        |
| cut $k$                 | $i + k + N$                             | $a$                                                            | $b + ak$                                                       |
| deal with increment $k$ | $modinv(k, N) \cdot i$                  | $a \cdot modinv(k, N)$                                         | $b$                                                            |

$a$ starts at 1, $b$ at 0. Use the input rules to apply the above, once, use $a \bmod N$ and $b \bmod N$ after every iteration, and in the end we have our values to plug into the calculation to skip the LCG steps and just calculate $i$ directly.


In [4]:
from typing import Tuple

a = b = k = N = int
ShuffleCalc = Callable[[a, b, k, N], Tuple[a, b]]


OPS_V: Dict[str, ShuffleCalc] = {
    "deal into new stack": lambda a, b, k, N: (-a, b - a),
    "cut": lambda a, b, k, N: (a, b + k * a),
    "deal with increment": lambda a, b, k, N: (a * pow(k, -1, N), b),
}


def skip_lcg(
    instructions: Sequence[str],
    deck_size: int = 119315717514047,
    repetitions: int = 101741582076661,
    ops: Dict[str, ShuffleCalc] = None,
) -> Callable[[int], int]:
    if ops is None:
        ops = OPS_V

    a, b = 1, 0
    for instruction in instructions:
        try:
            instr, num, *_ = re.split(r"\s*(-?\d+)", instruction)
            num = int(num)
        except ValueError:
            instr, num = instruction, 0
        a, b = ops[instr](a, b, num, deck_size)
        a, b = a % deck_size, b % deck_size

    # a^R mod N => pow(a, R, N), modinv(a, N) -> pow(a, -1, N)
    return lambda i, r=repetitions, n=deck_size: (
        (pow(a, r, n) * i + (1 - pow(a, r, n)) * pow(1 - a, -1, n) * b) % n
    )

In [5]:
print("Part 2:", skip_lcg(data.splitlines())(2020))

Part 2: 73394009116480
